In [1]:
import time

from collections import OrderedDict
from pathlib import Path

import torch
from lib.structures.field_list import collect

from lib import utils, logger, config, modeling, solver, data

import os
import sys
import lib.data.transforms2d as t2d
from PIL import Image
import numpy as np
from lib.utils.intrinsics import adjust_intrinsic

%load_ext autoreload
%autoreload 2

## Load Model

In [2]:
config.merge_from_file('configs/front3d_train_3d_test.yaml')

# inference settings
config.MODEL.FRUSTUM3D.IS_LEVEL_64 = False
config.MODEL.FRUSTUM3D.IS_LEVEL_128 = False
config.MODEL.FRUSTUM3D.IS_LEVEL_256 = False
config.MODEL.FRUSTUM3D.FIX = True

device = torch.device("cuda:0")
print("Load model...")
model = modeling.PanopticReconstruction()
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of Trainable Parameters: {}".format(pytorch_total_params))

model_dict = model.state_dict()
checkpoint = torch.load(config.MODEL.PRETRAIN)
pretrained_dict = checkpoint["model"]
model_dict.update(pretrained_dict)
# model.load_state_dict(checkpoint["model"])  # load model checkpoint
model.load_state_dict(model_dict) # Load pretrained parameters
model = model.to(device)  # move to gpu
model.switch_test()

Load model...
-----------------------------------------
unet_output_channels 16
unet_fetures 16
#params discriminator 173808
Number of Trainable Parameters: 29023809


## Load Config

In [3]:
from pathlib import Path
import lib.visualize as vis
from lib.structures.frustum import compute_camera2frustum_transform
from lib.structures import DepthMap

img_names = ["0001","0004","0006","0009","0018","0027","0030"]
for img_name in img_names:
    print("\nprocessing image: ", img_name)
    input_path = "data/front3d/70f7c6c1-c48f-4106-bcef-40b80b84bbad/rgb_{}.png".format(img_name)
    config.OUTPUT_DIR='output/00/{}/{}'.format(21,img_name)

    # Define image transformation.
    color_image_size = (320, 240)
    depth_image_size = (160, 120)

    imagenet_stats = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    image_transforms = t2d.Compose([
        t2d.Resize(color_image_size),
        t2d.ToTensor(),
        t2d.Normalize(imagenet_stats[0], imagenet_stats[1]),  # use imagenet stats to normalize image
    ])
    #Open and prepare input image.
    input_image = Image.open(input_path)
    input_image = image_transforms(input_image)
    input_image = input_image.unsqueeze(0).to(device)

    # Prepare intrinsic matrix.
    front3d_intrinsic = np.array(config.MODEL.PROJECTION.INTRINSIC)
    front3d_intrinsic = adjust_intrinsic(front3d_intrinsic, color_image_size, depth_image_size)
    front3d_intrinsic = torch.from_numpy(front3d_intrinsic).to(device).float()

    # Prepare frustum mask.
    front3d_frustum_mask = np.load(str("data/frustum_mask.npz"))["mask"]
    front3d_frustum_mask = torch.from_numpy(front3d_frustum_mask).bool().to(device).unsqueeze(0).unsqueeze(0)

    start = time.time()
    with torch.no_grad():
        results = model.inference(input_image, front3d_intrinsic, front3d_frustum_mask)
    end = time.time()
    print("Inference time: {}".format(end - start))
    print(f"Visualize results, save them at {config.OUTPUT_DIR}")
    
    
    ## Save imgs
    depth_map: DepthMap = results["depth"]

    # visualize results
    output_path = config.OUTPUT_DIR
    output_path = Path(output_path)
    output_path.mkdir(exist_ok=True, parents=True)

    # Save Meshes
    dense_dimensions = torch.Size([1, 1] + config.MODEL.FRUSTUM3D.GRID_DIMENSIONS)
    min_coordinates = torch.IntTensor([0, 0, 0]).to(device)
    truncation = config.MODEL.FRUSTUM3D.TRUNCATION
    iso_value = config.MODEL.FRUSTUM3D.ISO_VALUE

    geometry = results["frustum"]["geometry"]
    surface, _, _ = geometry.dense(dense_dimensions, min_coordinates, default_value=truncation)
    instances = results["panoptic"]["panoptic_instances"]
    semantics = results["panoptic"]["panoptic_semantics"]
    color = results["panoptic"]["rgb"]
    color = color.permute(1,2,3,0)

    camera2frustum = compute_camera2frustum_transform(depth_map.intrinsic_matrix.cpu(), torch.tensor(results["input"].size()) / 2.0,
                                                          config.MODEL.PROJECTION.DEPTH_MIN,
                                                          config.MODEL.PROJECTION.DEPTH_MAX,
                                                          config.MODEL.PROJECTION.VOXEL_SIZE)

    # remove padding: original grid size: [256, 256, 256] -> [231, 174, 187]
    camera2frustum[:3, 3] += (torch.tensor([256, 256, 256]) - torch.tensor([231, 174, 187])) / 2
    frustum2camera = torch.inverse(camera2frustum)
    vis.write_distance_field(surface.squeeze(), None, output_path / "mesh_geometry.ply", transform=frustum2camera)
    vis.write_distance_field(surface.squeeze(), color.squeeze(), output_path / "mesh_color.ply", transform=frustum2camera, is_color=True)
    vis.write_distance_field(surface.squeeze(), instances.squeeze(), output_path / "mesh_instances.ply", transform=frustum2camera)
    vis.write_distance_field(surface.squeeze(), semantics.squeeze(), output_path / "mesh_semantics.ply", transform=frustum2camera)


processing image:  0001


/usr/local/lib/python3.8/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/src/app/panoptic-reconstruction/lib/modeling/projection/sparse_projection.py:194: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  padding_offsets = difference // 2
/usr/local/lib/python3.8/dist-packages/MinkowskiEngine-0.5.1-py3.8-linux-x86_64.egg/MinkowskiEngine/MinkowskiSparseTensor.py:512: UserWarning: __floordiv__ is deprecated, and its behavior w

Inference time: 1.6115314960479736
Visualize results, save them at output/00/21/0001

processing image:  0004
Inference time: 1.018364429473877
Visualize results, save them at output/00/21/0004

processing image:  0006
Inference time: 0.8830006122589111
Visualize results, save them at output/00/21/0006

processing image:  0009
Inference time: 0.9789028167724609
Visualize results, save them at output/00/21/0009

processing image:  0018
Inference time: 0.9109153747558594
Visualize results, save them at output/00/21/0018

processing image:  0027
Inference time: 0.9248330593109131
Visualize results, save them at output/00/21/0027

processing image:  0030
Inference time: 0.9714713096618652
Visualize results, save them at output/00/21/0030
